In [11]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [12]:
# Metadata inladen
metadata = pd.read_csv("/Users/jortgommers/Desktop/Huidkanker-cnn-model-main/HAM10000_metadata.csv")
image_dir_part1 = "/Users/jortgommers/Desktop/Huidkanker-cnn-model-main/images/part_1"
image_dir_part2 = "/Users/jortgommers/Desktop/Huidkanker-cnn-model-main/images/part_2"


In [14]:
metadata['image_path'] = metadata['image_id'].apply(
    lambda x: os.path.join(
        image_dir_part1 if os.path.exists(os.path.join(image_dir_part1, f"{x}.jpg")) else image_dir_part2,
        f"{x}.jpg"
    )
)
metadata = metadata[metadata['image_path'].apply(os.path.exists)]


In [15]:
train, test = train_test_split(metadata, test_size=0.15, stratify=metadata['dx'], random_state=42)
train, val = train_test_split(train, test_size=0.15, stratify=train['dx'], random_state=42)


In [16]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
).flow_from_dataframe(
    train, x_col='image_path', y_col='dx', target_size=(224, 224),
    class_mode='categorical', batch_size=32
)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    val, x_col='image_path', y_col='dx', target_size=(224, 224),
    class_mode='categorical', batch_size=32
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    test, x_col='image_path', y_col='dx', target_size=(224, 224),
    class_mode='categorical', batch_size=32, shuffle=False
)

Found 7235 validated image filenames belonging to 7 classes.
Found 1277 validated image filenames belonging to 7 classes.
Found 1503 validated image filenames belonging to 7 classes.


In [17]:
# Model initialiseren
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
output = Dense(7, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)


In [18]:
# Mixed Precision Training
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)


In [19]:
# Compileer model
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [21]:
# Callbacks
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.2, patience=5, min_lr=1e-6)

In [22]:
# Model trainen
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=1, # 50
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    callbacks=[early_stopping, reduce_lr],  # ModelCheckpoint verwijderd
    verbose=1
)

/Users/jortgommers/opt/anaconda3/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  8/227 ━━━━━━━━━━━━━━━━━━━━ 22:47 6s/step - accuracy: 0.2895 - loss: 1.8158

KeyboardInterrupt: 

In [ ]:
test_loss, test_accuracy = model.evaluate(test_gen, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
# Classificatierapport en Confusion Matrix
predictions = np.argmax(model.predict(test_gen), axis=-1)
true_labels = test_gen.classes

print(classification_report(true_labels, predictions, target_names=list(test_gen.class_indices.keys())))
conf_matrix = confusion_matrix(true_labels, predictions)

In [ ]:
# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_gen.class_indices.keys(),
            yticklabels=test_gen.class_indices.keys())
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()



In [ ]:
# Opslaan van het model voor toekomstig gebruik
model.save('skin_cancer_classifier.h5')
